<h1 align=center> Segmenting and Clustering Neighborhoods in Toronto </h1>

<h2> Week 3 Part 1 Assignment </h2>
<h4>Part 1 of 3 Goal</h4>
<br><b>Tasks:</b> 
<ui>
    <li>Scrape Canada postal code information from wikipedia website.</li>
    <li>Load the data into a dataframe where data is wrangled, cleaned and transformed into a final dataframe.</li>
    <li>Populate the dataframe with Toronto Postal Codes, Borough and Neighborhood information.</li>
</ui>
<br> <br>Let's begin by building code to scrape following wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, to get our Postal Code, Borough, and Neighborhood data. Then we'll wrangle and clean the data.

In [2]:
# Load url into data objects for analysis: 
import pandas as pd #library for data analysis
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
print("Data Read from Wikipedia")

Data Read from Wikipedia


In [3]:
df = data[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Summary:

We retrieved all 3 JSON data objects from the wiki site. The first dataframe object data[0] created by html reader had the 3 column names we need. The data set is also populated with 180 rows of Toronto data.

data[1] and data[2] sets, our second and third imported dataframe objects we retrieved using read_html were evaluated and found to contain Canadian province/state higher-level domain information which is out of scope for our Toronto City level model needs. Therefore data[1] and data[2] sets are of no use to us.

Next, we ensure Borough (town) cells with an assigned borough are processed. Cells where borough = 'Not assigned' are ignored.

In [4]:
#Identify cell counts for each Borough. 
print(df.Borough.value_counts())

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
East Toronto         5
York                 5
Mississauga          1
Name: Borough, dtype: int64


Here above, we see 77 row counts found identified where Borough column has 'Not Assigned' row value.
Let's remove the 77 Borough cells with 'Not Assigned' values

In [5]:
# Remove 'Not assigned" count data on Borough
df = df[df['Borough'] != 'Not assigned']
print(df.Borough.value_counts())

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64


After applying our != operator above, the 77 'Not assigned' rows were successfully removed from Borough column.

Next task states when two or more Neighborhoods share the same postal code, we are to combine each Neighbourhood to a single neighborhood line that is comma delimited.

In [6]:
# I first perform a count sum of any df column duplicates by creating a lambda function. 
dup_chk = df.apply(lambda col: col.duplicated()).sum()
print(dup_chk)
print(df.head(5))

Postal Code       0
Borough          93
Neighbourhood     4
dtype: int64
  Postal Code           Borough                                Neighbourhood
2         M3A        North York                                    Parkwoods
3         M4A        North York                             Victoria Village
4         M5A  Downtown Toronto                    Regent Park, Harbourfront
5         M6A        North York             Lawrence Manor, Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government


The dup_chk lambda function above returned a 0 sum of Postal Code column duplicates, therefore no condition of two or more Neighborhoods sharing same Postal Code value exists within dataframe.
<br>
Neighbourhood column above shows comma delimited Neighbourhoods already getting combined to a single line as desired, where multiple neighbourhoods share same Postal Code.

In [7]:
#List all rows where Neighbourhood = 'Not assigned'. 
Num_Neighbourhood_Not_Assigneds = df[df['Neighbourhood'] == 'Not assigned']
Num_Neighbourhood_Not_Assigneds.head(5)

,Postal Code,Borough,Neighbourhood


No applicable 'Not assigned" Neighborhood cases found where Neighbourhood cells need to be assigned with Borough values.

Last step for Part 1, we use .shape method to print number of rows of dataframe

In [8]:
print(df.shape, 'rows x cols')

(103, 3) rows x cols


<h2> This concludes Week 3 Part 1 Assignment </h2>

<h2>Begin Week 3 Part 2 Assignment</h2>
<h4>Part 2 of 3 Goal</h4>
<br><b>Tasks:</b> 
<ui>
    <li>We add postal geolocation data to our existing postal dataframe with Neighbourhoods and Borough.</li>
    <li>We scrape Geospatial data for Toronto postal codes from "http://cocl.us/Geospatial_data".</li>
    <li>We merge the geodata into new dataframe joined on postal code.</li>
    <li>We conduct further analysis and data segmentation.</li>
</ui>
<br> <br>Let's begin by reading in geospatial data of Toronto postal codes from "http://cocl.us/Geospatial_data" and store into a new geocode dataframe.

In [9]:
#Load geocode information from http://cocl.us/Geospatial_data
geodata = pd.read_csv("http://cocl.us/Geospatial_data")
geodata.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<b>Postal Code</b> column is common in both <b>df</b> and <b>geodata</b> datafremes. We would crate a new dataframe <b>neigh_df</b> by merging <b>df</b> and <b>geodata</b> using <b>INNER JOIN</b>.

In [10]:
neigh_df = pd.merge(df,geodata, how='inner', on=None)
neigh_df.head(5)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Let's Validate the number of rows in newly created <b>neigh_df</b> and our initial dataframe <b>df</b>.
<br> Remember, the shape of <b>df</b> was <b>103 x 3</b> inidicating <b>103 rows</b> and <b>3 columns</b>.

In [11]:
print(neigh_df.shape)

(103, 5)


We see 103 lines of data, 5 expected columns, with Latitude and Longitude fully populated

<h2>This concludes Week 3 Part 2 Assignment</h2>

<h2>Begin Week 3 Part 2 Assignment</h2>
<h4>Part 3 of 3 Goal</h4>
<br><b>Tasks:</b> 
<ui>
    <li>Search for a specific venue category</li>
    <li>Explore Neighborhoods in Toronto</li>
    <li>Analyze Each Neighborhood</li>
    <li>Cluster Neighborhoods</li>
    <li>Examine Clusters</li>
</ui>
<br> <br>

In [12]:
import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes #uncomment this line geopy is not already installed
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes #Uncomment this line if Folium is not already installed.
import folium #map rendering library

print('Libraries imported.')

Libraries imported.


Validate the data in neigh_df dataframe.

In [13]:
neigh_df.head(5)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Access first Neighborhood <b>( location = 0 )</b>

In [14]:
neigh_df.loc[0,'Neighbourhood']

'Parkwoods'

Calculate the No. of Boroughs and Neighborhoods.

In [15]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neigh_df['Borough'].unique()),
        neigh_df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


Check Latitude and Longitude values in <b>neigh_df</b> dataframe for first row <b>( location = 0 )</b>.

In [16]:
neigh_df_latitude = neigh_df.loc[0, 'Latitude'] # neighborhood latitude value
neigh_df_longitude = neigh_df.loc[0, 'Longitude'] #neighborhood longitude value
neigh_df_neighborhood = neigh_df.loc[0,'Neighbourhood'] #neighborhood features

print('Latitude and longitude values of {} are {}, {}.'.format(neigh_df_latitude, 
                                                               neigh_df_longitude, 
                                                               neigh_df_neighborhood))

Latitude and longitude values of 43.7532586 are -79.3296565, Parkwoods.


<h3> Setting up Foursquare parameters </h3>

In [64]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


Check <b>Toronto</b> Geocodes.

In [18]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, ON are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, ON are 43.6534817, -79.3839347.


In [19]:
# For the search query, I set mid-downtown Toronto coordinates by calculating mean latitude and longitude 
# coordinates.  Alternative is to reference a physical address and city in the location field in next at next step.
meanlat = neigh_df.Latitude.mean()
meanlong = neigh_df.Longitude.mean()
MID_TORONTO = [meanlat, meanlong]

In [20]:
#We reference mean mid-toronto_downtown param above to derive geolocation as
#opposed to using the 'address' param
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(MID_TORONTO) #Defined from step above
latitude = location.latitude #uses mean mid-town location for latitude
longitude = location.longitude #uses mean mid-town location for longitude
print(latitude, longitude)

43.70468615 -79.39746395


Plot the map of Toronto using the above latitudes and longitudes.

In [21]:
# Next we create map of Toronto using latitude and longitude values and add markers to map and start exploring visuals.
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(neigh_df['Latitude'], neigh_df['Longitude'], 
                                           neigh_df['Borough'], neigh_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Neighbourhood names can be viewed by making mouse selection on markers.

<h3>1. Search for a specific venue category</h3>

> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

<br>Let's define a query to search for Italian food that is within 500 metres from the Conrad Hotel.

In [22]:
search_query = 'Take out'
print(search_query + ' .... OK!')

Take out .... OK!


Define the corresponding URL

In [65]:
radius=500
LIMIT=20
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=&client_secret=&ll=43.70468615,-79.39746395&v=20180604&query=Take out&radius=500&limit=20'

Send the GET Request and examine the results

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f33c2c5b0029033bdaf1a7e'},
 'response': {'venues': [{'id': '4e8c8f4d469079cdfae7fff9',
    'name': 'The Output',
    'location': {'address': '2180 Yonge Street',
     'crossStreet': 'Eglinton',
     'lat': 43.70459379087675,
     'lng': -79.39745177846619,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.70459379087675,
       'lng': -79.39745177846619}],
     'distance': 10,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['2180 Yonge Street (Eglinton)',
      'Toronto ON',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d144941735',
      'name': 'Caribbean Restaurant',
      'pluralName': 'Caribbean Restaurants',
      'shortName': 'Caribbean',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/caribbean_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1597227664',
    'hasPerk': False},
   {'id': '4f23180ee4b01b0c

Get relevant part of JSON and transform it into a *pandas* dataframe

In [25]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.name.head()

0                  The Output
1                 Checked Out
2    2180 Yonge Street Output
3               Indoors & Out
4      The Outdoor Break Room
Name: name, dtype: object

In [26]:
neigh_df.Borough.head(10)

0          North York
1          North York
2    Downtown Toronto
3          North York
4    Downtown Toronto
5           Etobicoke
6         Scarborough
7          North York
8           East York
9    Downtown Toronto
Name: Borough, dtype: object

Let's get the top 100 venues that are in North York within a radius of 500 meters.

In [66]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={},&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,'20200811',"North York",500,50)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=&client_secret=&v=20200811&near=North York,&radius=500&limit=50'

In [28]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f33c218b856fa49bb058ccd'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'geocode': {'what': '',
   'where': 'north york',
   'center': {'lat': 43.76681, 'lng': -79.4163},
   'displayString': 'North York, ON, Canada',
   'cc': 'CA',
   'geometry': {'bounds': {'ne': {'lat': 43.85873, 'lng': -79.296997},
     'sw': {'lat': 43.702629, 'lng': -79.557068}}},
   'slug': 'north-york-ontario-canada',
   'longId': '72057594044019040'},
  'headerLocation': 'North York',
  'headerFullLocation': 'North York',
  'headerLocationGranularity': 'city',
  'totalResults': 52,
  'suggestedBounds': {'ne': {'lat': 43.77078229753201,
    'lng': -79.41057980763519},
   'sw': {'lat': 43.76360295836879, 'lng': -79.41570705677928}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         

Let's borrow the <b>get_category_type</b> function from the Foursquare lab.

In [29]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the <b>json</b> and structure it into a <i>pandas</i> dataframe

In [30]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,The Keg,Steakhouse,43.766579,-79.412131
1,Konjiki Ramen,Ramen Restaurant,43.766998,-79.412222
2,Toronto Centre for the Arts,Theater,43.766228,-79.414115
3,Starbucks,Coffee Shop,43.768353,-79.413046
4,Saryo,Café,43.766979,-79.412205
5,Satay Sate,Indonesian Restaurant,43.766690,-79.412100
6,Mel Lastman Square,Plaza,43.767701,-79.412975
7,Loblaws,Grocery Store,43.768722,-79.412101
8,Sushi Moto Sake & Wine Bar,Sushi Restaurant,43.764095,-79.411344
9,Aroma Espresso Bar,Café,43.769449,-79.413081


Calculate <b>No. of Venues</b> returned by Foursquare API.

In [40]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

50 venues were returned by Foursquare.


<h3>2. Explore Neighborhoods in Toronto</h3>

Let's create a function to repeat the same process to all the neighborhoods in Midtown Toronto.
<br> Let's borrow <b>getNearbyVenues</b> function from the course lab.

In [67]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

nearby_venues

,name,categories,lat,lng
0,The Keg,Steakhouse,43.766579,-79.412131
1,Konjiki Ramen,Ramen Restaurant,43.766998,-79.412222
2,Toronto Centre for the Arts,Theater,43.766228,-79.414115
3,Starbucks,Coffee Shop,43.768353,-79.413046
4,Saryo,Café,43.766979,-79.412205
5,Satay Sate,Indonesian Restaurant,43.766690,-79.412100
6,Mel Lastman Square,Plaza,43.767701,-79.412975
7,Loblaws,Grocery Store,43.768722,-79.412101
8,Sushi Moto Sake & Wine Bar,Sushi Restaurant,43.764095,-79.411344
9,Aroma Espresso Bar,Café,43.769449,-79.413081


#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [42]:
toronto_venues = getNearbyVenues(names=neigh_df['Neighbourhood'], 
                latitudes=neigh_df['Latitude'], 
                longitudes=neigh_df['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

#### Let's check the size of the resulting dataframe

In [43]:
print(toronto_venues.shape)
toronto_venues.head()

(1068, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Sun Life,43.754760,-79.332783,Construction & Landscaping
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


#### Let's check how many venues were returned for each neighborhood

In [44]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",6,6,6,6,6,6
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",20,20,20,20,20,20
Berczy Park,20,20,20,20,20,20
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",20,20,20,20,20,20
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",18,18,18,18,18,18


<h3>3. Analyze Each Neighborhood</h3>

In [45]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Recording Studio,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Let's confirm the new size

In [46]:
toronto_onehot.shape

(1068, 214)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [47]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Recording Studio,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.250000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.2

#### Let's confirm the new size

In [48]:
toronto_grouped.shape

(95, 214)

#### Let's print each neighborhood along with the top 5 most common venues

In [49]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                        venue  freq
0   Latin American Restaurant  0.25
1                      Lounge  0.25
2                Skating Rink  0.25
3              Breakfast Spot  0.25
4  Modern European Restaurant  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.33
1             Gym  0.17
2             Pub  0.17
3  Sandwich Place  0.17
4     Coffee Shop  0.17


----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0                 Bank  0.10
1          Coffee Shop  0.10
2  Fried Chicken Joint  0.05
3     Sushi Restaurant  0.05
4          Supermarket  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.10
1  Italian Restaurant  0.10
2      Sandwich Place  0.10
3

Let's put this data into a dataframe.
<br>First, let's write a function to sort the venues in descending order (borrowing the function from the LAB).

In [50]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [51]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Skating Rink,Lounge,Breakfast Spot,Dance Studio,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Pub,Sandwich Place,Women's Store,Cuban Restaurant,Distribution Center,Discount Store,Diner
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Sushi Restaurant,Pharmacy,Pizza Place,Mobile Phone Shop,Middle Eastern Restaurant,Restaurant,Bridal Shop,Deli / Bodega
3,Bayview Village,Chinese Restaurant,Bank,Japanese Restaurant,Café,Women's Store,Dance Studio,Donut Shop,Dog Run,Distribution Center,Discount Store
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sandwich Place,Greek Restaurant,Thai Restaurant,Juice Bar,Liquor Store,Indian Restaurant,Restaurant,Sushi Restaurant


<h3> 4. Cluster Neighborhoods</h3>

Run *k*-means to cluster the neighborhood into 5 clusters.

In [52]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [53]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neigh_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
toronto_merged = toronto_merged[np.isfinite(toronto_merged['Cluster Labels'])]
toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Park,Construction & Landscaping,Food & Drink Shop,Women's Store,Dance Studio,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Pizza Place,Coffee Shop,Portuguese Restaurant,French Restaurant,Hockey Arena,Women's Store,Cuban Restaurant,Distribution Center,Discount Store,Diner
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2.0,Coffee Shop,Breakfast Spot,Bakery,Park,Farmers Market,Distribution Center,Pub,Historic Site,Dessert Shop,Spa
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2.0,Clothing Store,Accessories Store,Vietnamese Restaurant,Coffee Shop,Event Space,Boutique,Furniture / Home Store,Dance Studio,Dog Run,Distribution Center
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2.0,Coffee Shop,Yoga Studio,Arts & Crafts Store,Persian Restaurant,Portuguese Restaurant,Creperie,Mexican Restaurant,Diner,Distribution Center,Japanese Restaurant


Finally, let's visualize the resulting clusters

In [54]:
# create mapneigh
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 1

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Pizza Place,Coffee Shop,Portuguese Restaurant,French Restaurant,Hockey Arena,Women's Store,Cuban Restaurant,Distribution Center,Discount Store,Diner
50,North York,0,Pizza Place,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
63,York,0,Pizza Place,Grocery Store,Convenience Store,Bus Line,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
70,Etobicoke,0,Pizza Place,Coffee Shop,Chinese Restaurant,Discount Store,Intersection,Sandwich Place,Middle Eastern Restaurant,Curling Ice,Distribution Center,Diner
72,North York,0,Pharmacy,Bank,Pizza Place,Butcher,Coffee Shop,Grocery Store,Gastropub,Creperie,Distribution Center,Discount Store
77,Etobicoke,0,Pizza Place,Sandwich Place,Bus Line,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
93,Etobicoke,0,Pizza Place,Coffee Shop,Gym,Pub,Sandwich Place,Women's Store,Cuban Restaurant,Distribution Center,Discount Store,Diner


#### Cluster 2

In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,York,1,Park,Women's Store,Pool,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
35,East York,1,Park,Intersection,Convenience Store,Women's Store,Dance Studio,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
52,North York,1,Park,Women's Store,Curling Ice,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
64,York,1,Park,Women's Store,Curling Ice,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
66,North York,1,Park,Convenience Store,Women's Store,Curling Ice,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
85,Scarborough,1,Park,Playground,Women's Store,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
91,Downtown Toronto,1,Park,Playground,Trail,Women's Store,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant


#### Cluster 3

In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2,Park,Construction & Landscaping,Food & Drink Shop,Women's Store,Dance Studio,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
2,Downtown Toronto,2,Coffee Shop,Breakfast Spot,Bakery,Park,Farmers Market,Distribution Center,Pub,Historic Site,Dessert Shop,Spa
3,North York,2,Clothing Store,Accessories Store,Vietnamese Restaurant,Coffee Shop,Event Space,Boutique,Furniture / Home Store,Dance Studio,Dog Run,Distribution Center
4,Downtown Toronto,2,Coffee Shop,Yoga Studio,Arts & Crafts Store,Persian Restaurant,Portuguese Restaurant,Creperie,Mexican Restaurant,Diner,Distribution Center,Japanese Restaurant
7,North York,2,Gym,Japanese Restaurant,Restaurant,Coffee Shop,Beer Store,Art Gallery,Supermarket,Sporting Goods Shop,Café,Italian Restaurant
8,East York,2,Pizza Place,Pharmacy,Café,Breakfast Spot,Intersection,Bank,Athletics & Sports,Pet Store,Gym / Fitness Center,Gastropub
9,Downtown Toronto,2,Café,Shopping Mall,Electronics Store,Sporting Goods Shop,Burger Joint,Ramen Restaurant,Burrito Place,Sandwich Place,Comic Shop,Plaza
10,North York,2,Park,Japanese Restaurant,Pub,Metro Station,Women's Store,Curling Ice,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
12,Scarborough,2,Bar,Women's Store,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
13,North York,2,Gym,Japanese Restaurant,Restaurant,Coffee Shop,Beer Store,Art Gallery,Supermarket,Sporting Goods Shop,Café,Italian Restaurant


#### Cluster 4

In [61]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,3,Fast Food Restaurant,Women's Store,Curling Ice,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop


#### Cluster 5

In [63]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,4,Baseball Field,Women's Store,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
62,Central Toronto,4,Home Service,Garden,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
101,Etobicoke,4,Baseball Field,Home Service,Women's Store,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner


<h2>This concludes Week 3 Part 3 Assignment</h2>